## Authenticate and Initialize Earth Engine

This cell authenticates to Google Earth Engine using a service account and initializes the `ee` Python API for further operations.

In [1]:
import ee
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    'ee-lmfphks8-00182c921204.json',
    scopes=['https://www.googleapis.com/auth/earthengine']
)
ee.Initialize(credentials=credentials)

## Check Asset Metadata via REST API

This cell uses the REST API and your access token to fetch and print the metadata for a specific Earth Engine asset.

In [2]:
import requests

token = credentials.token

# API CHECK 
url = "https://earthengine.googleapis.com/v1alpha/projects/earthengine-public/assets/COPERNICUS/S2/20170430T190351_20170430T190351_T10SEG"
headers = {"Authorization": f"Bearer {token}"}
response = requests.get(url, headers=headers)
print(response.json())

{'type': 'IMAGE', 'name': 'projects/earthengine-public/assets/COPERNICUS/S2/20170430T190351_20170430T190351_T10SEG', 'id': 'COPERNICUS/S2/20170430T190351_20170430T190351_T10SEG', 'properties': {'DATATAKE_IDENTIFIER': 'GS2A_20170430T190351_009691_N02.05', 'SPACECRAFT_NAME': 'Sentinel-2A', 'FORMAT_CORRECTNESS_FLAG': 'PASSED', 'IERS_BULLETIN_FILENAME': 'S2__OPER_AUX_UT1UTC_PDMC_20170427T000000_V20170428T000000_20180427T000000', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 114.909983716, 'MEAN_SOLAR_AZIMUTH_ANGLE': 144.170082506, 'SOLAR_IRRADIANCE_B12': 85.25, 'SOLAR_IRRADIANCE_B10': 367.15, 'SENSOR_QUALITY': 'PASSED', 'SOLAR_IRRADIANCE_B11': 245.59, 'GENERATION_TIME': 1493579031000, 'SOLAR_IRRADIANCE_B8A': 955.19, 'FORMAT_CORRECTNESS': 'PASSED', 'PRODUCT_URI': 'S2A_MSIL1C_20170430T190351_N0205_R113_T10SEG_20170430T190351.SAFE', 'SENSOR_QUALITY_FLAG': 'PASSED', 'CLOUD_COVERAGE_ASSESSMENT': 0, 'DATASTRIP_ID': 'S2A_OPER_MSI_L1C_DS_MPS__20170430T234718_S20170430T190351_N02.05', 'PROCESSING_BASELINE': 

## Fetch Asset Metadata from croissant.json URL

This cell loads a URL from `croissant.json`, uses your Earth Engine token to fetch the asset metadata, and prints the result.

In [3]:
import json
import requests
from google.oauth2 import service_account
from google.auth.transport.requests import Request

credentials = service_account.Credentials.from_service_account_file(
    'ee-lmfphks8-00182c921204.json',
    scopes=['https://www.googleapis.com/auth/earthengine']
)

credentials.refresh(Request())

token = credentials.token

with open("croissant.json", "r") as f:
    croissant_metadata = json.load(f)

asset_url = croissant_metadata['url']  # URL extracted

headers = {"Authorization": f"Bearer {token}"}

response = requests.get(asset_url, headers=headers)

if response.status_code == 200:
    asset_metadata = response.json()
    print("Asset Metadata Retrieved Successfully!")
    print(json.dumps(asset_metadata, indent=2)) 
else:
    print(f"Error fetching metadata. HTTP status code: {response.status_code}")

Asset Metadata Retrieved Successfully!
{
  "type": "IMAGE",
  "name": "projects/earthengine-public/assets/COPERNICUS/S2/20170430T190351_20170430T190351_T10SEG",
  "id": "COPERNICUS/S2/20170430T190351_20170430T190351_T10SEG",
  "properties": {
    "DATATAKE_IDENTIFIER": "GS2A_20170430T190351_009691_N02.05",
    "SPACECRAFT_NAME": "Sentinel-2A",
    "FORMAT_CORRECTNESS_FLAG": "PASSED",
    "IERS_BULLETIN_FILENAME": "S2__OPER_AUX_UT1UTC_PDMC_20170427T000000_V20170428T000000_20180427T000000",
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A": 114.909983716,
    "MEAN_SOLAR_AZIMUTH_ANGLE": 144.170082506,
    "SOLAR_IRRADIANCE_B12": 85.25,
    "SOLAR_IRRADIANCE_B10": 367.15,
    "SENSOR_QUALITY": "PASSED",
    "SOLAR_IRRADIANCE_B11": 245.59,
    "GENERATION_TIME": 1493579031000,
    "SOLAR_IRRADIANCE_B8A": 955.19,
    "FORMAT_CORRECTNESS": "PASSED",
    "PRODUCT_URI": "S2A_MSIL1C_20170430T190351_N0205_R113_T10SEG_20170430T190351.SAFE",
    "SENSOR_QUALITY_FLAG": "PASSED",
    "CLOUD_COVERAGE_ASSESSMENT

## Convert Earth Engine Asset Metadata to GeoCroissant JSON-LD

This cell:
- Authenticates to Earth Engine,
- Fetches asset metadata,
- Computes the bounding box and WKT geometry,
- Builds a per-band asset dictionary,
- Assembles a GeoCroissant-compliant JSON-LD object,
- Saves it to `gee.json`.

In [4]:
import ee
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import json
import datetime

# 1. Authenticate to Earth Engine
SERVICE_ACCOUNT_FILE = "ee-lmfphks8-00182c921204.json"
ASSET_ID = "COPERNICUS/S2/20170430T190351_20170430T190351_T10SEG"

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=["https://www.googleapis.com/auth/earthengine"]
)
creds.refresh(Request())
ee.Initialize(credentials=creds)
token = creds.token

# 2. Fetch asset metadata
meta = ee.data.getAsset(ASSET_ID)
props = meta["properties"]

# 3. Compute bounding box
coords = meta["geometry"]["coordinates"][0]
lons, lats = zip(*coords)
bbox = f"{min(lats)} {min(lons)} {max(lats)} {max(lons)}"
wkt = "POLYGON((" + ", ".join(f"{x} {y}" for x, y in coords) + "))"

# 4. Build per-band assets
assets = {}
for band in meta["bands"]:
    band_id = band["id"]
    res = band["grid"]["affineTransform"]["scaleX"]
    assets[band_id] = {
        "href": f"ee://{ASSET_ID}/{band_id}",
        "type": "image/tiff",
        "roles": ["data"],
        "band_name": band_id,
        "data_type": band["dataType"]["precision"].lower(),
        "spatial_resolution": res,
        "description": f"Sentinel-2 band {band_id} of image {ASSET_ID}"
    }

# 5. Assemble Geo-Croissant JSON-LD (using correct prefixes & geocr IRIs)
geo_croissant = {
    "@context": {
        "@vocab": "https://schema.org/",
        "sc": "https://schema.org/",
        "cr": "http://mlcommons.org/croissant/",
        "geocr": "http://mlcommons.org/croissant/geocr/",
        "dct": "http://purl.org/dc/terms/"
    },
    "@type": "sc:Dataset",
    # name sanitized (no slashes)
    "sc:name": ASSET_ID.replace("/", "-"),
    "sc:description": (
        f"Sentinel-2 Level-1C image over MGRS tile {props.get('MGRS_TILE','')}"
        f" acquired on {meta['startTime'][:10]}"
    ),
    "sc:identifier": f"projects/earthengine-public/assets/{ASSET_ID}",
    "sc:license": "https://creativecommons.org/licenses/by/4.0/",
    "sc:datePublished": datetime.date.today().isoformat(),
    "sc:dateModified": meta.get("updateTime"),
    "dct:conformsTo": "http://mlcommons.org/croissant/1.0",
    "sc:temporalCoverage": f"{meta['startTime']}/{meta['endTime']}",
    "geocr:BoundingBox": bbox,
    "geocr:Geometry": {
        "@type": "geocr:Geometry",
        "geocr:asWKT": wkt
    },
    "geocr:measurementType": f"{props.get('SPACECRAFT_NAME','')} MSI",
    "cr:variableMeasured": [
        {
            "@type": "sc:PropertyValue",
            "sc:name": "Cloudy pixel percentage",
            "sc:value": props.get("CLOUDY_PIXEL_PERCENTAGE")
        },
        {
            "@type": "sc:PropertyValue",
            "sc:name": "Cloud coverage assessment",
            "sc:value": props.get("CLOUD_COVERAGE_ASSESSMENT")
        }
    ],
    # STAC-style assets block is allowed as Geo-Croissant extension
    "assets": assets,
    # Recommended schema.org props to silence warnings
    "sc:citation": f"https://earthengine.googleapis.com/v1alpha/projects/earthengine-public/assets/{ASSET_ID}",
    "sc:version": "1.0"
}

# 6. Write to file
with open("gee.json", "w") as f:
    json.dump(geo_croissant, f, indent=2)

print("Geo-Croissant JSON-LD saved to gee.json")

Geo-Croissant JSON-LD saved to gee.json


## Pretty Print GeoCroissant JSON-LD

This cell loads and pretty-prints the contents of `gee.json` for easy inspection.

In [5]:
import json

# Load and pretty-print the content of croissant.json
with open("gee.json", "r") as f:
    croissant_data = json.load(f)

# Pretty-print JSON to console
print(json.dumps(croissant_data, indent=2))

{
  "@context": {
    "@vocab": "https://schema.org/",
    "sc": "https://schema.org/",
    "cr": "http://mlcommons.org/croissant/",
    "geocr": "http://mlcommons.org/croissant/geocr/",
    "dct": "http://purl.org/dc/terms/"
  },
  "@type": "sc:Dataset",
  "sc:name": "COPERNICUS-S2-20170430T190351_20170430T190351_T10SEG",
  "sc:description": "Sentinel-2 Level-1C image over MGRS tile 10SEG acquired on 2017-04-30",
  "sc:identifier": "projects/earthengine-public/assets/COPERNICUS/S2/20170430T190351_20170430T190351_T10SEG",
  "sc:license": "https://creativecommons.org/licenses/by/4.0/",
  "sc:datePublished": "2025-07-23",
  "sc:dateModified": "2025-01-17T23:27:34.445839Z",
  "dct:conformsTo": "http://mlcommons.org/croissant/1.0",
  "sc:temporalCoverage": "2017-04-30T19:03:51.460Z/2017-04-30T19:03:51.460Z",
  "geocr:BoundingBox": "36.95157206201084 -123.00034179205349 37.94767956582544 -121.75075919661815",
  "geocr:Geometry": {
    "@type": "geocr:Geometry",
    "geocr:asWKT": "POLYGON((

## Validate GeoCroissant JSON-LD

This cell runs the `mlcroissant` validator on your GeoCroissant JSON-LD file to check for schema compliance.

In [6]:
!mlcroissant validate --jsonld=geocroissant_single_image.json

W0723 11:02:22.178388 137841581138304 rdf.py:80] WARNING: The JSON-LD `@context` is not standard. Refer to the official @context (e.g., from the example datasets in https://github.com/mlcommons/croissant/tree/main/datasets/1.0). The different keys are: {'path', 'fileProperty', 'parentField', 'references', 'separator', 'fileObject', 'recordSet', 'repeated', 'transform', 'dataType', 'data', 'fileSet', 'key', 'isLiveDataset', 'jsonPath', '@language', 'rai', 'column', 'format', 'regex', 'md5', 'examples', 'field', 'citeAs', 'subField', 'replace', 'conformsTo', 'includes', 'extract', 'source'}
W0723 11:02:22.195979 137841581138304 datasets.py:41] Found the following 1 warning(s) during the validation:
  -  [Metadata(COPERNICUS-S2-20170430T190351_20170430T190351_T10SEG)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.
I0723 11:02:22.196084 137841581138304 validate.py:53] Done.


## Visualize Earth Engine Asset and Thumbnails

This cell:
- Loads the GeoCroissant JSON-LD,
- Extracts the Earth Engine asset ID,
- Authenticates and initializes Earth Engine,
- Loads the image and centers a map,
- Adds true-color and additional band layers,
- Generates and displays RGB and band thumbnails,
- Displays an interactive map using geemap.

In [8]:
import ee
import json
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import geemap.foliumap as geemap
from IPython.display import display
from ipywidgets import HBox, Image
import requests

# 1. Load Geo-Croissant JSON-LD
with open("gee.json") as f:
    md = json.load(f)

# 2. Extract EE asset ID
parts = md["sc:identifier"].split("/")
ee_id = "/".join(parts[-3:])  # e.g. "COPERNICUS/S2/20170430T190351_20170430T190351_T10SEG"

# 3. Authenticate & init EE
SERVICE_ACCOUNT_FILE = "ee-lmfphks8-00182c921204.json"
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=["https://www.googleapis.com/auth/earthengine"]
)
creds.refresh(Request())
ee.Initialize(credentials=creds)

# 4. Load image and center map
img = ee.Image(ee_id)
centroid = img.geometry().centroid().coordinates().getInfo()[::-1]  # [lat, lon]
m = geemap.Map(center=centroid, zoom=10)

# 5. Add true-color composite (B4-3-2) and two extra bands
m.addLayer(img, {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000, "gamma": 1.2},
           "True Color (4-3-2)")
for b in ["B8", "B11"]:
    m.addLayer(img.select(b), {"min": 0, "max": 3000}, f"Band {b}")

# 6a. Generate an inline RGB composite thumbnail
rgb_url = img.getThumbURL({
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 3000,
    "dimensions": 256,
    "region": img.geometry()
})
resp = requests.get(rgb_url)
rgb_thumb = Image(value=resp.content, format='png', width=128, height=128)

# 6b. Generate thumbnails for other bands (optional)
band_list = ["B8", "B11"]
thumbs = []
for b in band_list:
    band_url = img.select(b).getThumbURL({
        "min": 0,
        "max": 3000,
        "dimensions": 256,
        "region": img.geometry()
    })
    band_resp = requests.get(band_url)
    band_thumb = Image(value=band_resp.content, format='png', width=128, height=128)
    thumbs.append(band_thumb)

# 7. Display band thumbs plus the RGB thumb
display(HBox([rgb_thumb] + thumbs))

# 8. Display the interactive map
m